# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [91]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

# improved memory usage see
# https://py.checkio.org/blog/memory-optimization-with-python-slots/ 
# this is due to dataclasses and python classes use not optimized dicts under the hood
# by default and @dataclass(slots=True) provides information in __slots__ dunder field
# on how much space we need for instance of class. Not huge optimization, but something
# useful to do in any case.
@dataclass(slots=True)
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)

    __rmul__ = (
        __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    )
    __radd__ = (
        __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
    )


def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d


Поддерживаются две операции - сложение и умножение. Применить можно так:

In [92]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

As for optimization as I wrote in initial version we can optimize memory usage
(which I suppose should be done all the time except some corner cases when we
know that `__slots__` would be incompatible).

To check differentiation correctness we can check if for symmetry and asymmetry,
how it handles constants, power operations, etc.

As for pow, unary, division operations see computation graph-based
implementation beneath.

In [93]:
tests = [
    ("lambda x: x * x", diff(lambda x: x * x), 2, 4, -4),
    ("lambda x: x + 10", diff(lambda x: x + 10), 2, 1, 1),
    ("lambda x: x + (-1) * x * x", diff(lambda x: x + (-1) * x * x), 2, -3, 5),
]

for s, f, p, v, av in tests:
    assert f(p) == v, f"Failed: {s}, {p}, {v}"
    assert f(-p) == av, f"Failed: {s}, -{p}, {av}"

print("Success")

Success


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

Solution based on computational graph below.

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [94]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

# Solution

Fundamental idea is to build computational graph, which resembles the concept of
Tensor Flow (and neural networks in general). We build graph, where nodes are
some operation, symbolic variables or constant. Then going forward through graph
we can compute solution for some argument and going backward we can compute
diff.

So:
- forward propagation (looping through graph in topological order) = compute the
  value
- backward propagation (looping though graph in reverse topological order) =
  compute derivatives

More on computational graphs:

- https://www.tutorialspoint.com/python_deep_learning/python_deep_learning_computational_graphs.htm
- https://www.geeksforgeeks.org/computational-graphs-in-deep-learning/
- https://towardsdatascience.com/evolution-of-graph-computation-and-machine-learning-3211e8682c83

In [147]:
import typing
import dataclasses
import math
import random
import ast

import numpy as np
import sympy
import mpmath


In [96]:
def _cast_to_node(v: "Node | float | int") -> "Node":
    match v:
        case Node() as n:
            return n
        case _:
            return Con(v)

@typing.runtime_checkable
class Node(typing.Protocol):
    def backward(self, n: "Node") -> "Node":
        ...

    def forward(self) -> float | int:
        ...

    
    def __add__(self, other):
        return Sum(self, _cast_to_node(other))

    def __radd__(self, other):
        return Sum(_cast_to_node(other), self)

    def __mul__(self, other):
        return Mul(self, _cast_to_node(other))

    def __rmul__(self, other):
        return Mul(_cast_to_node(other), self)

    def __neg__(self):
        return Neg(self)

    def __sub__(self, other):
        return Sum(self, Neg(_cast_to_node(other)))

    def __rsub__(self, other):
        return Sum(_cast_to_node(other), Neg(self))

    def __pow__(self, other):
        return Pow(self, _cast_to_node(other))

    def __rpow__(self, other):
        return Pow(_cast_to_node(other), self)

    def __truediv__(self, other):
        return Div(self, _cast_to_node(other))

    def __rtruediv__(self, other):
        return Div(_cast_to_node(other), self)

    def __abs__(self):
        return Abs(self)


`Node` is base `Protocol` for nodes of computational graph. Method `forward`
does forward propagation (value computation in our case). Method `backward`
intuitively means backward propagation and diff computation.

So generally we have to provide the following types:

```mermaid
graph TD
    Node
    
    subgraph Values
        Variable
        Const
    end

    subgraph Operations
        Sum
        Neg
        Mul
        Div
        Pow
        Log
        Cos
        Sin
        Exp
        Abs
    end

    Node-->Variable
    Node-->Const
    Node-->Sum
    Node-->Neg
    Node-->Mul
    Node-->Div
    Node-->Pow
    Node-->Log
    Node-->Cos
    Node-->Sin
    Node-->Exp
    Node-->Abs
```

`dataclass(slots=True)` decorator is used for `Node` `Protocol` implementation
for memory improvement.

Provided implementation supports multi-argument functions by default (task 1.5).

In [97]:
@dataclass(slots=True)
class Con(Node):

    value: int | float

    def backward(self, _) -> Node:
        return Con(0.0)

    def forward(self) -> float | int:
        return self.value

    def __repr__(self):
        return str(self.value)


In [98]:
@dataclass(slots=True)
class Var(Node):

    name: str
    value: int | float | None

    def backward(self, n: Node) -> Node:
        return Con(1) if self == n else Con(0)

    def forward(self) -> float | int:
        if self.value is None:
            raise ValueError("Empty variable")

        return self.value

    def __repr__(self):
        return f"{self.name}"


In [99]:
@dataclass(slots=True)
class Sum(Node):

    x: Node
    y: Node

    def backward(self, n: Node) -> Node:
        return Sum(self.x.backward(n), self.y.backward(n))

    def forward(self) -> float | int:
        return self.x.forward() + self.y.forward()

    def __repr__(self):
        return f"({self.x} + {self.y})"


In [100]:
@dataclass(slots=True)
class Mul(Node):

    x: Node
    y: Node

    def backward(self, n: Node) -> Node:
        return Sum(Mul(self.x.backward(n), self.y), Mul(self.x, self.y.backward(n)))

    def forward(self) -> float | int:
        return self.x.forward() * self.y.forward()

    def __repr__(self):
        return f"({self.x} * {self.y})"


In [101]:
@dataclass(slots=True)
class Neg(Node):

    x: Node

    def backward(self, n: Node) -> Node:
        return Neg(self.x.backward(n))

    def forward(self) -> float | int:
        return -self.x.forward()

    def __repr__(self):
        return f"(-{self.x})"


In [114]:
@dataclass(slots=True)
class Pow(Node):

    x: Node
    y: Node

    def backward(self, n: Node) -> Node:
        return Sum(
            Mul(
                Mul(self.y, Pow(self.x, Sum(self.y, Con(-1)))), self.x.backward(n)
            ),
            Mul(Mul(Pow(self.x, self.y), self.y.backward(n)), Log(self.x)),
        )

    def forward(self) -> int | float:
        return self.x.forward() ** self.y.forward()

    def __repr__(self):
        return f"({self.x} ^ {self.y})"


In [103]:
from operator import neg


@dataclass(slots=True)
class Div(Node):

    x: Node
    y: Node

    def backward(self, n: Node) -> Node:
        return Div(
            Sum(
                Mul(self.x.backward(n), self.y),
                Neg(Mul(self.x, self.y.backward(n))),
            ),
            Pow(self.y, Con(2)),
        )

    def forward(self) -> float | int:
        return self.x.forward() / self.y.forward()

    def __repr__(self):
        return f"({self.x} / {self.y})"


In [104]:
@dataclass(slots=True)
class Abs(Node):

    x: Node

    def backward(self, n: Node) -> Node:
        return Abs(self.x.backward(n))

    def forward(self) -> float | int:
        match v := self.x.forward():
            case _ if v >= 0:
                return v
            case _:
                return -v
        

    def __repr__(self):
        return f"(|{self.x}|)"


Task 1.2 solution

In [105]:
@dataclass(slots=True)
class Log(Node):

    x: Node

    def backward(self, n: Node) -> Node:
        return Mul(Div(Con(1), self.x), self.x.backward(n))

    def forward(self) -> float | int:
        return np.log(self.x.forward())

    def __repr__(self):
        return f"log({self.x})"


In [106]:
@dataclass(slots=True)
class Exp(Node):

    x: Node

    def backward(self, n: Node) -> Node:
        return Mul(Exp(self.x), self.x.backward(n))

    def forward(self) -> float | int:
        return np.exp(self.x.forward())

    def __repr__(self):
        return f"e^({self.x})"


In [107]:
@dataclass(slots=True)
class Cos(Node):

    x: Node

    def backward(self, n: Node) -> Node:
        return Mul(Neg(Sin(self.x)), self.x.backward(n))

    def forward(self) -> float | int:
        return np.cos(self.x.forward())

    def __repr__(self):
        return f"cos({self.x})"


In [108]:
@dataclass(slots=True)
class Sin(Node):

    x: Node

    def backward(self, n: Node) -> Node:
        return Mul(Cos(self.x), self.x.backward(n))

    def forward(self) -> float | int:
        return np.sin(self.x.forward())

    def __repr__(self):
        return f"sin({self.x})"


Checking differentiation against [`sympy`](https://www.sympy.org/en/index.html)
a library for symbolic computation and very simple Newton's difference qoutient
with a little modification (see
[question](https://stackoverflow.com/questions/41833759/finding-the-difference-quotient-of-a-function-using-python)).

Inspired by https://stackoverflow.com/questions/9876290/how-do-i-compute-derivative-using-numpy

One-argument tests.

In [125]:
def dfb(f, x, h):
    hh = 0.5 * h 
    return (f(x + hh) - f(x - hh)) / h

In [138]:
value = np.random.randint(1, 50)
x = Var("x", value)
sx = sympy.Symbol("x")
single_tests = [
    (
        (lambda z: z + z**2 - 7),
        (x + x**2 - 7),
        (sx + sx**2 - 7),
    ),
    (
        (lambda z: z**3),
        (x**3),
        (sx**3),
    ),
    (
        (lambda z: 3 / z**2),
        (3 / x**2),
        (3 / sx**2),
    ),
    (
        (lambda z: np.log(np.sin(z))),
        (Log(Sin(x))),
        (sympy.log(sympy.sin(sx))),
    ),
    (
        (lambda z: np.cos(z) - np.sin(z)),
        (Cos(x) - Sin(x)),
        (sympy.cos(sx) - sympy.sin(sx)),
    ),
    (
        (lambda z: np.exp(z) - z),
        (Exp(x) - x),
        (sympy.exp(sx) - sx),
    ),
]

for lf, f, sf in single_tests:
    l_r = dfb(lf, value, 0.0001)
    f_r = f.backward(x).forward()
    s_r = sympy.lambdify([sx], sympy.diff(sf, sx))(value)
    print(f_r, s_r, l_r)
    assert math.isclose(f_r, s_r,rel_tol=0.000000001)
    assert math.isclose(f_r, l_r,rel_tol=0.000000001)
    print(f"Success {f} against {sf} and simple Newton's difference quotient")

print("\nSuccess")


17.0 17 16.999999999995907
Success ((x + (x ^ 2)) + (-7)) against x**2 + x - 7 and simple Newton's difference quotient
192.0 192 192.0000000023947
Success (x ^ 3) against x**3 and simple Newton's difference quotient
-0.01171875 -0.01171875 -0.011718750000874745
Success (3 / (x ^ 2)) against 3/x**2 and simple Newton's difference quotient
-0.1470650639494805 -0.1470650639494805 -0.14706506407360237
Success log(sin(x)) against log(sin(x)) and simple Newton's difference quotient
-0.8438582128147682 -0.8438582128147682 -0.8438582124625782
Success (cos(x) + (-sin(x))) against -sin(x) + cos(x) and simple Newton's difference quotient
2979.9579870417283 2979.9579870417283 2979.957988272872
Success (e^(x) + (-x)) against -x + exp(x) and simple Newton's difference quotient

Success


Multi-argument tests

In [139]:
x_value, y_value = np.random.randint(1, 50), np.random.randint(1, 50)
x , y = Var("x", x_value), Var("y", y_value)
sx , sy = sympy.Symbol("x"), sympy.Symbol("y")

multi_tests = [
    (
        (x + y),
        (sx + sy)
    ),
    (
        (x - y),
        (sx - sy),
    ),
    (
        (x * y + x),
        (sx * sy + sx),
    ),
    (
        ((x + y) / x),
        ((sx + sy) / sx),
    ),
    (
        (x ** 3 + y ** x),
        (sx ** 3 + sy ** sx)
    ),
    (
        (Cos(x) - Log(y)),
        (sympy.cos(sx) - sympy.log(sy)),
    ),
    (
        (Exp(x) / Sin(y)),
        (sympy.exp(sx) / sympy.sin(sy)),
    )
]

for f, sf in multi_tests:
    f_r = f.backward(x).forward()
    s_r = sympy.lambdify([sx, sy], sympy.diff(sf, sx))(x_value, y_value)
    print(f_r, s_r)
    assert math.isclose(f_r, s_r)
    print(f"Success {f} against {sf} on x")
    
    f_r = f.backward(y).forward()
    s_r = sympy.lambdify([sx, sy], sympy.diff(sf, sy))(x_value, y_value)
    print(f_r, s_r)
    assert math.isclose(f_r, s_r)
    print(f"Success {f} against {sf} on y")

print("Success")

1 1
Success (x + y) against x + y on x
1 1
Success (x + y) against x + y on y
1 1
Success (x + (-y)) against x - y on x
-1 -1
Success (x + (-y)) against x - y on y
39 39
Success ((x * y) + x) against x*y + x on x
15 15
Success ((x * y) + x) against x*y + x on y
-0.1688888888888889 -0.16888888888888887
Success ((x + y) / x) against (x + y)/x on x
0.06666666666666667 0.06666666666666667
Success ((x + y) / x) against (x + y)/x on y
1.8095360433494654e+24 1.8095360433494654e+24
Success ((x ^ 3) + (y ^ x)) against x**3 + y**x on x
1.963638830980016e+23 1.963638830980016e+23
Success ((x ^ 3) + (y ^ x)) against x**3 + y**x on y
-0.6502878401571168 -0.6502878401571168
Success (cos(x) + (-log(y))) against -log(y) + cos(x) on x
-0.02631578947368421 -0.02631578947368421
Success (cos(x) + (-log(y))) against -log(y) + cos(x) on y
11030242.769688688 11030242.76968869
Success (e^(x) / sin(y)) against exp(x)/sin(y) on x
-35545921.23985947 -35545921.23985947
Success (e^(x) / sin(y)) against exp(x)/sin(

For task 1.5 based on `ast` let's build generator for random function.

In [160]:
def _arg_name(i: int):
    return f"x{i}"


def _random_expr(args_amount: int, recursion_limit: int):
    recursion_limit -= 1

    if recursion_limit == 1:
        unary_node_without_recursive_calls_type = random.choice(
            [
                "const",
                "var",
            ]
        )
        if unary_node_without_recursive_calls_type == "const":
            value = random.randint(-10, 10)
            return [
                ast.Call(
                    ast.Name(id="Con", ctx=ast.Load()),
                    args=[ast.Constant(value)],
                    keywords=[],
                ),
                ast.Constant(value),
            ]
        elif unary_node_without_recursive_calls_type == "var":
            name = _arg_name(random.randint(0, args_amount - 1))
            return [
                ast.Name(id=name, ctx=ast.Load()),
                ast.Name(id=name, ctx=ast.Load()),
            ]

    # choose binary or unary node
    if random.random() < 0.4:
        # expression will be bin op
        left = _random_expr(args_amount, recursion_limit)
        right = _random_expr(args_amount, recursion_limit)
        op = random.choice([ast.Add(), ast.Sub(), ast.Mult(), ast.Div(), ast.Pow()])
        return [
            ast.BinOp(left=left[0], op=op, right=right[0]),
            ast.BinOp(left=left[1], op=op, right=right[1]),
        ]
    else:
        # expression will be single node
        unary_node_type = random.choice(
            [
                "const",
                "var",
                "abs",
                "cos",
                "sin",
                "exp",
                "log",
            ]
        )
        if unary_node_type == "const":
            value = random.randint(-10, 10)
            return [
                ast.Call(
                    ast.Name(id="Con", ctx=ast.Load()),
                    args=[ast.Constant(value)],
                    keywords=[],
                ),
                ast.Constant(value),
            ]
        if unary_node_type == "var":
            name = _arg_name(random.randint(0, args_amount - 1))
            return [
                ast.Name(id=name, ctx=ast.Load()),
                ast.Name(id=name, ctx=ast.Load()),
            ]
        elif unary_node_type == "abs":
            expr = _random_expr(args_amount, recursion_limit)
            return [
                ast.Call(
                    ast.Name(id="Abs", ctx=ast.Load()), args=[expr[0]], keywords=[]
                ),
                ast.Call(
                    ast.Name(id="abs", ctx=ast.Load()), args=[expr[1]], keywords=[]
                ),
            ]
        elif unary_node_type == "cos":
            expr = _random_expr(args_amount, recursion_limit)
            return [
                ast.Call(
                    ast.Name(id="Cos", ctx=ast.Load()), args=[expr[0]], keywords=[]
                ),
                ast.Call(
                    ast.Name(id="cos", ctx=ast.Load()), args=[expr[1]], keywords=[]
                ),
            ]
        elif unary_node_type == "sin":
            expr = _random_expr(args_amount, recursion_limit)
            return [
                ast.Call(
                    ast.Name(id="Sin", ctx=ast.Load()), args=[expr[0]], keywords=[]
                ),
                ast.Call(
                    ast.Name(id="sin", ctx=ast.Load()), args=[expr[1]], keywords=[]
                ),
            ]
        elif unary_node_type == "log":
            expr = _random_expr(args_amount, recursion_limit)
            return [
                ast.Call(
                    ast.Name(id="Log", ctx=ast.Load()), args=[expr[0]], keywords=[]
                ),
                ast.Call(
                    ast.Name(id="log", ctx=ast.Load()), args=[expr[1]], keywords=[]
                ),
            ]
        elif unary_node_type == "exp":
            expr = _random_expr(args_amount, recursion_limit)
            return [
                ast.Call(
                    ast.Name(id="Exp", ctx=ast.Load()), args=[expr[0]], keywords=[]
                ),
                ast.Call(
                    ast.Name(id="exp", ctx=ast.Load()), args=[expr[1]], keywords=[]
                ),
            ]


def generate_function(
    args_amount: int = 1, recursion_limit: int = 10, verbose=False
) -> Callable[[float], float]:
    body = _random_expr(args_amount, recursion_limit)
    expr = ast.Expression(
        body=ast.Lambda(
            args=ast.arguments(
                args=[ast.arg(arg=_arg_name(i)) for i in range(args_amount)],
                posonlyargs=[],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[],
            ),
            body=body[0],
        )
    )

    expr_for_num_diff = ast.Expression(
        body=ast.Lambda(
            args=ast.arguments(
                args=[ast.arg(arg=_arg_name(i)) for i in range(args_amount)],
                posonlyargs=[],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[],
            ),
            body=body[1],
        )
    )

    ast.fix_missing_locations(expr)
    ast.fix_missing_locations(expr_for_num_diff)
    if verbose:
        print(f"Generated function for auto-diff: {ast.unparse(expr)}")
        print(
            f"Generated function for numerical diff: {ast.unparse(expr_for_num_diff)}\n"
        )
    compiled_func = compile(expr, filename="", mode="eval")
    compiled_func_for_num_diff = compile(expr_for_num_diff, filename="", mode="eval")
    func = eval(
        compiled_func,
        {"Abs": Abs, "Cos": Cos, "Sin": Sin, "Log": Log, "Exp": Exp, "Con": Con},
    )
    func_for_numerical_diff = eval(
        compiled_func_for_num_diff,
        {
            "abs": abs,
            "cos": math.cos,
            "sin": math.sin,
            "log": math.log,
            "exp": math.exp,
        },
    )
    return func, func_for_numerical_diff


In [188]:
for _ in range(5):
    value = np.random.randint(1, 50)
    x = Var('x', value)
    f_to_cg, lf = generate_function(recursion_limit=4)
    f = f_to_cg(x)

    print(f"f(x) = {f}")
    print(f"f'(x) = {f.backward(x)}")
    f_r = f.backward(x).forward()
    dfb_r = dfb(lf, value, 0.00001)

    print(f_r, dfb_r)
    assert not np.isnan(f_r) or math.isclose(f_r, dfb_r)
    print('---------------------------------------------------------------------------')

f(x) = ((3 / x) + (-(3 + -7)))
f'(x) = ((((0.0 * x) + (-(3 * 1))) / (x ^ 2)) + (-(0.0 + 0.0)))
-0.12 -0.1199999999812462
---------------------------------------------------------------------------
f(x) = (-7 / (-2 + (-9)))
f'(x) = (((0.0 * (-2 + (-9))) + (-(-7 * (0.0 + (-0.0))))) / ((-2 + (-9)) ^ 2))
0.0 0.0
---------------------------------------------------------------------------
f(x) = x
f'(x) = 1
1 0.9999999999621422
---------------------------------------------------------------------------
f(x) = (sin(x) + (|x|))
f'(x) = ((cos(x) * 1) + (|1|))
0.8720363103725953 0.8720363105396699
---------------------------------------------------------------------------
f(x) = (sin(-6) + (-(2 / x)))
f'(x) = ((cos(-6) * 0.0) + (-(((0.0 * x) + (-(2 * 1))) / (x ^ 2))))
0.0010330578512396695 0.001033057853150332
---------------------------------------------------------------------------
